# Parameters
in this cell the used variables can be changed 

height and width correspond to the dimensions of the Workspace, in which the Topology Optimisation is going to be performed

log_dir is the direction where the current best Model will be saved to during training

ts is the amount of timesteps the model will be trained for




In [1]:
import numpy as np
import gymnasium as gym

import autograd.numpy as anp

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter

In [2]:
height = 13
width = 19
log_dir = "log"
ts = 1e6

density = 1e-4
constraint_thresh = 0.7


normals = np.zeros((height, width, 2))
x = 0
y = 1
print("normalsX\n" + str(normals[:,:,x]))
print("normalsY\n" + str(normals[:,:,y]))



number_of_nodes = height * width

normalsX
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
normalsY
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0

Here the reward for each state of the workspace is the compliance
which should be minimized or above a certain threshold



In [3]:
# TODO write the function to get the reward
# It should return the compliance of the structure
# The compliance is the sum of the forces applied to the structure
# multiplied by the displacement of the nodes
# The goal of the Optimization will be to minimize the compliance or keep it below a certain threshold


def rewardFunction(state):
    return 1




def constraintFunction(state):
    return 1

In [4]:
class TopOptEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self):
        # The constructor of the environment

        super().__init__()
        self.design_space = anp.ones((height, width))
        self.bound_nodes = np.zeros((height, width))
        self.force_nodes = np.zeros((height, width))
        self.density_matrix = self.design_space * density
        

        self.action_space = gym.spaces.Discrete(number_of_nodes)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(height, width), dtype=np.float64)

        # A Dictionary is used to map each coordinate tuple of the designspace to
        # a singular distinct integer for use in the 
        k=0
        for i in range(height):
            for j in range(width):
                self._actions_to_coordinates[k] = (i,j)
                k += 1


        print("self.n:\t" + str(self.n) + "\nself.m:\t" + str(self.m))
        self._actions_to_coordinates = {} 
        self.reward = 0
        self.step_count = 0

        self.reset()

    def step(self, action):
        #
        # TODO finisch the step function for the environment	
        # we need to add the reward function and the termination condition
        #
        #
        # action is the coordinate of the node that is going to be changed
        # action is a number between 0 and number_of_nodes

        self.step_count += 1
        terminated = False

        #check if the selected Node is either bound or force node
        if self.bound_nodes[self._actions_to_coordinates[action]] == 1:
            self.reward = -1
            return self.design_space, self.reward, terminated, False, self._getInfo()
        
        if self.force_nodes[self._actions_to_coordinates[action]] == 1:
            self.reward = -1
            return self.design_space, self.reward, terminated, False, self._getInfo()
        
        if self.design_space[self._actions_to_coordinates[action]] == 0:
            self.reward = -1
            return self.design_space, self.reward, terminated, False, self._getInfo()

        # exectute an action and get the Reward
        self.design_space[self._actions_to_coordinates[action]] = 0
        self.reward = rewardFunction(self.design_space)
        self.constraint = constraintFunction(self.design_space)

        # 
        if self.constraint > constraint_thresh:
            self.reward -= 1

        
        if self.step_count > number_of_nodes:
            terminated = True

        



        return self.design_space, self.reward, terminated, False, self._getInfo()
    
    
    def reset(self, seed=None):
        # Reset the environment
        # This function returns the initial state of the environment
        # The initial state should be a 2D array of size (height, width)
        #


        super().reset(seed=seed)
        self.design_space = anp.ones((height, width))
        self.density_matrix = self.design_space * density 

        self.constraint = constraintFunction(self.design_space)
        self.reward = 0
        self.step_count = 0

        #initialize the bound and force nodes
        self.bound_nodes = np.zeros((height, width))
        self.force_nodes = np.zeros((height, width))
        
        # TODO: think about how force and bound nodes should be set
        # should both be randomized on each reset?
        # and if so how should they be randomized?

        return self.design_space, self._getInfo()
    

    
    def _getInfo(self):
        # This function returns the information about the environment
        # This function is used to monitor the environment
        # The information should be a dictionary
        # The dictionary should contain the following keys:
        # - step_count: the number of steps that have been executed
        # - current Reward: the reward of the current state
        # - design_space: the current state of the environment
        return {"step_count": self.step_count, 
                "current Reward": self.reward,
                "design_space": self.design_space}
    
    def debug(self):
        print(self.design_space)

In [5]:
env = TopOptEnv()
check_env(env, warn=True)
env.debug()

AttributeError: 'TopOptEnv' object has no attribute 'n'